# OsuLearn
##### Machine Learning para jogar mapas de osu!

^^^

Isso aí, tô sem mais nada pra fazer, bora criar uma IA que joga osu! )o)

## Importar umas coisas...

In [1]:
# Machine Learning
import tensorflow as tf
import keras
import numpy as np
import pandas as pd

# Plotagem
import matplotlib.pyplot as plt

# Utilidades
import os
import math

# Lógica do osu!
import osu.beatmap
import osu.replay

import osulearn

c:\users\user\appdata\local\programs\python\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html


## Constantes

Aqui tem uns caminhos para os arquivos que precisamos:

In [2]:
# Pasta do osu!
OSU_FOLDER = "C:\\" + os.path.join("Program Files (x86)", "Jogos", "osu!")

## Dados para treinamento...

In [3]:
import osu.replay
import re

from glob import glob, escape

def load_replay_files():
    replay_training_set = []
    pattern = os.path.join(OSU_FOLDER, "Replays", "* Osu.osr")
    for replay_file in glob(pattern):
        info = re.search(r"([^\\/]+) \- (.+ \- .+) \[(.+)\] \((.+)\)", replay_file)
        player, bmap, diff, date = info[1], info[2], info[3], info[4]
        print(player, bmap, diff, date)

        fpat = os.path.join(OSU_FOLDER, "Songs", "**", "*" + escape(bmap) + "*" + escape("[" + diff + "]") + ".osu")

        beatmap_file = glob(fpat)
        if len(beatmap_file) > 0:
            beatmap = osu.beatmap.load(beatmap_file[0])
            if re.search("v14", beatmap.format_version) != None:
                replay = osu.replay.load(replay_file)

                if not (replay.mods & 0x10 or replay.mods & 0x40):
                    replay_training_set.append([beatmap, replay])
                    print("OK", len(replay_training_set) - 1)
                else:
                    print("Replay uses DT or HR. Those mods are not supported yet.")
            else:
                print("Wrong format version")
        else:
            print("Beatmap file not found")
        print()
        
    return replay_training_set

In [4]:
from importlib import reload
reload(osulearn)
reload(osu.replay)

try:
    X = np.load('input_data.npy')
    y = np.load('target_data.npy')
    
except:
    try:
        replay_training_set
    except NameError:
        replay_training_set = load_replay_files()
    
    print("Processing Input Data...")
    X = osulearn.create_training_data(replay_training_set)
    print()
    
    print("Processing Target Data...")
    y = osulearn.create_target_data(replay_training_set)

    np.save('input_data.npy', X)
    np.save('target_data.npy', y)

print(X.shape, y.shape)

BzMasked (K)NoW_NAME - Freesia Expert 2018-12-30
OK 0

BzMasked 3R2 - Bunny Panic!!! Bunny Extinction 2018-12-30
OK 1

BzMasked 55x55 - MRAZ' (feat. Larin) Insane 2018-12-30
OK 2

BzMasked Aimer with chelly (EGOIST) - ninelie (REDSHiFT x Vesuvia remix) Daydream 2018-06-01
OK 3

BzMasked Aitsuki Nakuru - Monochrome Butterfly Extra 2017-09-09
OK 4

BzMasked Aitsuki Nakuru - Monochrome Butterfly Mysterious 2017-08-04
OK 5

BzMasked Alex Gaudino feat. Crystal Waters - Destination Calabria Destination Unknown 2017-11-25
OK 6

BzMasked Alex Gaudino feat. Crystal Waters - Destination Calabria Destination Unknown 2017-12-01
OK 7

BzMasked Amidst - Phantasy NYA! 2018-12-30
OK 8

BzMasked Asterisk - Ren-chon no Drum 'n' Bass Extra 2018-12-30
OK 9

BzMasked Ata - Euphoria Juggernaut 285bpm Tap jumps 2017-09-24
OK 10

BzMasked Ayase Rie - Yuima-ruWorld TVver. Ultimate 2017-08-14
OK 11

BzMasked Ayase Rie - Yuima-ruWorld TVver. Ultimate 2018-12-30
OK 12

BzMasked Ayumi. - Hanagoyomi short version t

Exit This Earth's Atomosphere
Lone Digger
Weaponized
More One Night (Assertive Hardcore Bootleg) [short ver.]
Nanairo Symphony -TV Size-
Nanairo Symphony -TV Size-
Ghost Rule
Kanata No Reflesia
Allegro Furioso
Allegro Furioso
The Pink Fields
A Grande Familia (TV Size)
Stay Alive
-BWW SCREAM-
furioso melodia
Friendly Shop
Zen Zen Zense
Bang Bang
Bang Bang
Harumachi Clover
Harumachi Clover
Uraomote Lovers
Kawaiku Naritai (Short Ver.)
Coloring
True truly love
None
Transform (Original Mix)
Make a Move (Speed Up Ver.)
Warriors
Uta
Yomi yori Kikoyu, Koukoku no Tou to Honoo no Shoujo.
Fatima (TV Size)
Hammer o Denpa Song ni Shitemita
Sukisuki Zecchoushou
STARRING STAR
Hated By Life Itself.
Heart no Atoaji
Heart no Atoaji
Caught Fire (feat. Ashley Lawless)
Leave It All To Me
Milk Crown on Sonnetica
Amatsu Kitsune
Toaru Shoukoku no Ohimesama ga...
Ibuki
confusion
Highscore
ILY
A Young Girl, So Vivid in the Night
No title
sweets parade
sweets parade
Six Trillion Years and Overnight Story
Imprint

## Modelo de rede neural

Agora começa a festa...

In [5]:
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Activation, CuDNNLSTM, concatenate, Input, Reshape, Conv1D, UpSampling1D

REPLAY_FEATURES = ['x', 'y']
MAP_FEATURES = ['x', 'y', 'time_left', 'is_slider', 'is_spinner']

model = Sequential()

map_input = Input(shape=(X.shape[1], X.shape[2]), name='map_info')

out = CuDNNLSTM(len(MAP_FEATURES), return_sequences=True)(map_input)
out = Conv1D(256, X.shape[1], activation='linear')(out)
out = Dropout(0.1)(out)

out = Dense(256, activation='linear')(out)
out = Dropout(0.1)(out)

#out = Dense(128, activation='linear')(out)
#out = Dropout(0.2)(out)

out = Dense(X.shape[1] * len(REPLAY_FEATURES), activation='linear')(out)
out = Dropout(0.1)(out)

out = Reshape([X.shape[1], len(REPLAY_FEATURES)])(out)

model = Model(inputs=map_input, outputs=out)

model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
map_info (InputLayer)        (None, 2000, 5)           0         
_________________________________________________________________
cu_dnnlstm_1 (CuDNNLSTM)     (None, 2000, 5)           240       
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 1, 256)            2560256   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1, 256)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 1, 256)            65792     
_________________________________________________________________
dropout_2 (Dropout)          (None, 1, 256)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 1, 4000)           1028000   
__________

In [6]:
def plot_info(compare):
    %matplotlib inline
    
    for data in compare:
        plt.plot([(x) for x, y in data])
    plt.show()
    
    for data in compare:
        plt.plot([(y) for x, y in data])
    plt.show()
    
#     for data in compare:
#         plt.plot([(a) for distance, angle in data])
#     plt.show()
    
#     for data in compare:
#         plt.plot([(b) for distance, angle in data])
    plt.show()

In [8]:
from random import randint
from sklearn.model_selection import train_test_split

ITERATIONS = 16
EPOCHS = 64

try:
    acc
    loss
except NameError:
    acc = []
    loss = []
    
#model.load_weights("model.hdf5")
for i in range(ITERATIONS):
    print("-" * 80)
    print("Iteration #%d" % (i + 1))
    print("-" * 80)
    print()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=randint(0, 100))
    h = model.fit(X_train, y_train, batch_size=len(X_train), epochs=EPOCHS, verbose=1)
    acc += h.history['acc']
    loss += h.history['loss']
    print()

model.save_weights("model.hdf5")

--------------------------------------------------------------------------------
Iteration #1
--------------------------------------------------------------------------------

Epoch 1/64
412/412 [==============================] - 4s 10ms/step - loss: 0.0494 - acc: 0.5016
Epoch 2/64
412/412 [==============================] - 0s 683us/step - loss: 0.0503 - acc: 0.5132
Epoch 3/64
412/412 [==============================] - 0s 645us/step - loss: 0.0528 - acc: 0.5096
Epoch 4/64
412/412 [==============================] - 0s 683us/step - loss: 0.0503 - acc: 0.5159
Epoch 5/64
412/412 [==============================] - 0s 683us/step - loss: 0.0497 - acc: 0.5230
Epoch 6/64
412/412 [==============================] - 0s 645us/step - loss: 0.0495 - acc: 0.5278
Epoch 7/64
412/412 [==============================] - 0s 721us/step - loss: 0.0493 - acc: 0.5319
Epoch 8/64
412/412 [==============================] - 0s 607us/step - loss: 0.0491 - acc: 0.5365
Epoch 9/64
412/412 [=============================

Epoch 18/64
412/412 [==============================] - 0s 645us/step - loss: 0.0223 - acc: 0.7633
Epoch 19/64
412/412 [==============================] - 0s 702us/step - loss: 0.0221 - acc: 0.7640
Epoch 20/64
412/412 [==============================] - 0s 645us/step - loss: 0.0217 - acc: 0.7657
Epoch 21/64
412/412 [==============================] - 0s 651us/step - loss: 0.0214 - acc: 0.7668
Epoch 22/64
412/412 [==============================] - 0s 607us/step - loss: 0.0212 - acc: 0.7680
Epoch 23/64
412/412 [==============================] - 0s 645us/step - loss: 0.0210 - acc: 0.7711
Epoch 24/64
412/412 [==============================] - 0s 645us/step - loss: 0.0206 - acc: 0.7741
Epoch 25/64
412/412 [==============================] - 0s 645us/step - loss: 0.0204 - acc: 0.7746
Epoch 26/64
412/412 [==============================] - 0s 607us/step - loss: 0.0203 - acc: 0.7729
Epoch 27/64
412/412 [==============================] - 0s 683us/step - loss: 0.0201 - acc: 0.7733
Epoch 28/64
412/412 

Epoch 36/64
412/412 [==============================] - 0s 607us/step - loss: 0.0151 - acc: 0.8034
Epoch 37/64
412/412 [==============================] - 0s 690us/step - loss: 0.0151 - acc: 0.8058
Epoch 38/64
412/412 [==============================] - 0s 645us/step - loss: 0.0151 - acc: 0.8026
Epoch 39/64
412/412 [==============================] - 0s 683us/step - loss: 0.0152 - acc: 0.8034
Epoch 40/64
412/412 [==============================] - 0s 607us/step - loss: 0.0150 - acc: 0.8053
Epoch 41/64
412/412 [==============================] - 0s 645us/step - loss: 0.0149 - acc: 0.8056
Epoch 42/64
412/412 [==============================] - 0s 683us/step - loss: 0.0149 - acc: 0.8041
Epoch 43/64
412/412 [==============================] - 0s 683us/step - loss: 0.0150 - acc: 0.8059
Epoch 44/64
412/412 [==============================] - 0s 645us/step - loss: 0.0149 - acc: 0.8076
Epoch 45/64
412/412 [==============================] - 0s 645us/step - loss: 0.0147 - acc: 0.8053
Epoch 46/64
412/412 

Epoch 54/64
412/412 [==============================] - 0s 679us/step - loss: 0.0135 - acc: 0.8146
Epoch 55/64
412/412 [==============================] - 0s 645us/step - loss: 0.0136 - acc: 0.8178
Epoch 56/64
412/412 [==============================] - 0s 645us/step - loss: 0.0135 - acc: 0.8108
Epoch 57/64
412/412 [==============================] - 0s 645us/step - loss: 0.0136 - acc: 0.8049
Epoch 58/64
412/412 [==============================] - 0s 648us/step - loss: 0.0136 - acc: 0.8173
Epoch 59/64
412/412 [==============================] - 0s 645us/step - loss: 0.0134 - acc: 0.8119
Epoch 60/64
412/412 [==============================] - 0s 645us/step - loss: 0.0135 - acc: 0.8022
Epoch 61/64
412/412 [==============================] - 0s 645us/step - loss: 0.0135 - acc: 0.8153
Epoch 62/64
412/412 [==============================] - 0s 720us/step - loss: 0.0135 - acc: 0.8151
Epoch 63/64
412/412 [==============================] - 0s 607us/step - loss: 0.0135 - acc: 0.8072
Epoch 64/64
412/412 

Epoch 7/64
412/412 [==============================] - 0s 645us/step - loss: 0.0148 - acc: 0.8041
Epoch 8/64
412/412 [==============================] - 0s 607us/step - loss: 0.0147 - acc: 0.8078
Epoch 9/64
412/412 [==============================] - 0s 645us/step - loss: 0.0145 - acc: 0.8079
Epoch 10/64
412/412 [==============================] - 0s 645us/step - loss: 0.0145 - acc: 0.8069
Epoch 11/64
412/412 [==============================] - 0s 607us/step - loss: 0.0143 - acc: 0.8080
Epoch 12/64
412/412 [==============================] - 0s 683us/step - loss: 0.0143 - acc: 0.8129
Epoch 13/64
412/412 [==============================] - 0s 683us/step - loss: 0.0141 - acc: 0.8149
Epoch 14/64
412/412 [==============================] - 0s 683us/step - loss: 0.0141 - acc: 0.8130
Epoch 15/64
412/412 [==============================] - 0s 683us/step - loss: 0.0140 - acc: 0.8113
Epoch 16/64
412/412 [==============================] - 0s 645us/step - loss: 0.0138 - acc: 0.8107
Epoch 17/64
412/412 [==

...

Epoch 14/64
412/412 [==============================] - 0s 645us/step - loss: 0.0127 - acc: 0.8131
Epoch 15/64
412/412 [==============================] - 0s 683us/step - loss: 0.0128 - acc: 0.8195
Epoch 16/64
412/412 [==============================] - 0s 645us/step - loss: 0.0128 - acc: 0.8139
Epoch 17/64
412/412 [==============================] - 0s 663us/step - loss: 0.0127 - acc: 0.8133
Epoch 18/64
412/412 [==============================] - 0s 645us/step - loss: 0.0126 - acc: 0.8168
Epoch 19/64
412/412 [==============================] - 0s 645us/step - loss: 0.0127 - acc: 0.8219
Epoch 20/64
412/412 [==============================] - 0s 683us/step - loss: 0.0126 - acc: 0.8157
Epoch 21/64
412/412 [==============================] - 0s 607us/step - loss: 0.0126 - acc: 0.8106
Epoch 22/64
412/412 [==============================] - 0s 645us/step - loss: 0.0125 - acc: 0.8183
Epoch 23/64
412/412 [==============================] - 0s 645us/step - loss: 0.0124 - acc: 0.8204
Epoch 24/64
412/412 

Epoch 32/64
412/412 [==============================] - 0s 758us/step - loss: 0.0122 - acc: 0.8174
Epoch 33/64
412/412 [==============================] - 0s 629us/step - loss: 0.0122 - acc: 0.8135
Epoch 34/64
412/412 [==============================] - 0s 683us/step - loss: 0.0121 - acc: 0.8218
Epoch 35/64
412/412 [==============================] - 0s 683us/step - loss: 0.0121 - acc: 0.8231
Epoch 36/64
412/412 [==============================] - 0s 645us/step - loss: 0.0122 - acc: 0.8122
Epoch 37/64
412/412 [==============================] - 0s 607us/step - loss: 0.0122 - acc: 0.8077
Epoch 38/64
412/412 [==============================] - 0s 720us/step - loss: 0.0121 - acc: 0.8227
Epoch 39/64
412/412 [==============================] - 0s 645us/step - loss: 0.0121 - acc: 0.8216
Epoch 40/64
412/412 [==============================] - 0s 645us/step - loss: 0.0121 - acc: 0.8090
Epoch 41/64
412/412 [==============================] - 0s 683us/step - loss: 0.0119 - acc: 0.8187
Epoch 42/64
412/412 

Epoch 50/64
412/412 [==============================] - 0s 607us/step - loss: 0.0120 - acc: 0.8184
Epoch 51/64
412/412 [==============================] - 0s 645us/step - loss: 0.0121 - acc: 0.8193
Epoch 52/64
412/412 [==============================] - 0s 683us/step - loss: 0.0122 - acc: 0.8176
Epoch 53/64
412/412 [==============================] - 0s 645us/step - loss: 0.0121 - acc: 0.8173
Epoch 54/64
412/412 [==============================] - 0s 645us/step - loss: 0.0120 - acc: 0.8167
Epoch 55/64
412/412 [==============================] - 0s 645us/step - loss: 0.0122 - acc: 0.8179
Epoch 56/64
412/412 [==============================] - 0s 683us/step - loss: 0.0121 - acc: 0.8165
Epoch 57/64
412/412 [==============================] - 0s 645us/step - loss: 0.0121 - acc: 0.8172
Epoch 58/64
412/412 [==============================] - 0s 645us/step - loss: 0.0120 - acc: 0.8202
Epoch 59/64
412/412 [==============================] - 0s 645us/step - loss: 0.0121 - acc: 0.8169
Epoch 60/64
412/412 

412/412 [==============================] - 0s 683us/step - loss: 0.0133 - acc: 0.8145
Epoch 3/64
412/412 [==============================] - 0s 645us/step - loss: 0.0133 - acc: 0.8202
Epoch 4/64
412/412 [==============================] - 0s 647us/step - loss: 0.0132 - acc: 0.8142
Epoch 5/64
412/412 [==============================] - 0s 683us/step - loss: 0.0131 - acc: 0.8117
Epoch 6/64
412/412 [==============================] - 0s 721us/step - loss: 0.0130 - acc: 0.8144
Epoch 7/64
412/412 [==============================] - 0s 607us/step - loss: 0.0130 - acc: 0.8180
Epoch 8/64
412/412 [==============================] - 0s 683us/step - loss: 0.0129 - acc: 0.8151
Epoch 9/64
412/412 [==============================] - 0s 645us/step - loss: 0.0128 - acc: 0.8135
Epoch 10/64
412/412 [==============================] - 0s 683us/step - loss: 0.0127 - acc: 0.8187
Epoch 11/64
412/412 [==============================] - 0s 645us/step - loss: 0.0126 - acc: 0.8210
Epoch 12/64
412/412 [==================

412/412 [==============================] - 0s 683us/step - loss: 0.0123 - acc: 0.8221
Epoch 21/64
412/412 [==============================] - 0s 645us/step - loss: 0.0124 - acc: 0.8165
Epoch 22/64
412/412 [==============================] - 0s 607us/step - loss: 0.0124 - acc: 0.8166
Epoch 23/64
412/412 [==============================] - 0s 683us/step - loss: 0.0125 - acc: 0.8214
Epoch 24/64
412/412 [==============================] - 0s 683us/step - loss: 0.0124 - acc: 0.8225
Epoch 25/64
412/412 [==============================] - 0s 683us/step - loss: 0.0123 - acc: 0.8160
Epoch 26/64
412/412 [==============================] - 0s 645us/step - loss: 0.0122 - acc: 0.8202
Epoch 27/64
412/412 [==============================] - 0s 607us/step - loss: 0.0123 - acc: 0.8217
Epoch 28/64
412/412 [==============================] - 0s 683us/step - loss: 0.0123 - acc: 0.8223
Epoch 29/64
412/412 [==============================] - 0s 683us/step - loss: 0.0122 - acc: 0.8155
Epoch 30/64
412/412 [===========

In [ ]:
%matplotlib inline

plt.plot(acc)
plt.show()

plt.plot(loss)
plt.show()

In [ ]:
predicted = model.predict(X)

for i in range(100):#len(predicted)):
    print(i)
    plot_info([y[i], predicted[i]])

In [ ]:
# %matplotlib notebook

# replay_data = []

# n = 35

# j = 0
# for i in range(len(predicted[n])):
#     if len(replay_training_set[n][1].data) > i + j:
#         w = replay_training_set[n][1].data[i + j][0]
#         while w < 0 and len(replay_training_set[n][1].data) > i + j + 1:
#             j += 1
#             w = replay_training_set[n][1].data[i + j][0]
#     else:
#         w = 0
        
#     cx, cy = predicted[n][i]
#     replay_data.append([w, cx * 512 + 256, cy * 384 + 192, 0])
    
# osu.replay.preview(replay_training_set[n][0], [replay_data, replay_training_set[n][1].data])
# #osu.replay.preview(replay_training_set[n][0], replay_training_set[n][1].data)